In [1]:
import os
import time
import numpy as np
import pandas as pd
import random
import math
from pylab import rcParams
rcParams['figure.figsize'] = 20, 15

# import TrajectoryImprovementLibrary. as TIL

import plotly.graph_objs as go
import plotly.express as px

from sklearn.preprocessing import MinMaxScaler

import folium

In [2]:
trajectory1 = np.load( os.getcwd() + '/SampleTraj/Trj_0.npy' )
trajectory1 = pd.DataFrame(trajectory1)
trajectory1.columns = ['timestamp', 'latitude', 'longitude', 'altitude']
trajectory1 = trajectory1.drop_duplicates(subset=['latitude', 'longitude'], keep='first').to_numpy()


trajectory2 = np.load( os.getcwd() + '/SampleTraj/Trj_1.npy' )
trajectory2 = pd.DataFrame(trajectory2)
trajectory2.columns = ['timestamp', 'latitude', 'longitude', 'altitude']
trajectory2 = trajectory2.drop_duplicates(subset=['latitude', 'longitude'], keep='first').to_numpy()


trajectory3 = np.load( os.getcwd() + '/SampleTraj/Trj_2.npy' )
trajectory3 = pd.DataFrame(trajectory3)
trajectory3.columns = ['timestamp', 'latitude', 'longitude', 'altitude']
trajectory3 = trajectory3.drop_duplicates(subset=['latitude', 'longitude'], keep='first').to_numpy()

trajectory4 = np.load( os.getcwd() + '/SampleTraj/Trj_3.npy' )
trajectory4 = pd.DataFrame(trajectory4)
trajectory4.columns = ['timestamp', 'latitude', 'longitude', 'altitude']
trajectory4 = trajectory4.drop_duplicates(subset=['latitude', 'longitude'], keep='first').to_numpy()
# targetPoint = -88

In [3]:
auxParameters = { 'numIters': 1 }

In [4]:
def getNeighbours( point ):
    neighbours = []

    for _ in range( 500 ):

        neighbour = np.copy( point )

        neighbour[1] += np.random.normal( 0, .1 )
        neighbour[2] += np.random.normal( 0, .1 )

        neighbours.append(neighbour)

    return np.array(neighbours)

In [5]:
# def aux_CalculateTrajectoryError( trajectory, variablePoints ):
    
#     totalError = 0.0
    
#     for k in range( np.size( variablePoints ) ):
        
#         if variablePoints[ k ]:
#             single_point_error = aux_CalculateErrorSinglePoint( trajectory, k )
            
#             if ~np.isnan(single_point_error):

#                 totalError += single_point_error
    
#     return totalError




# def aux_CalculateErrorSinglePoint( trajectory, targetPoint ):
    
    
#     v_m1 = aux_haversine( trajectory[ targetPoint - 1, 2 ], trajectory[ targetPoint - 1, 1 ], trajectory[ targetPoint, 2 ], trajectory[ targetPoint, 1 ] )
#     s_m1 = v_m1 / ( trajectory[ targetPoint, 0 ] - trajectory[ targetPoint - 1, 0 ] )
    
    
#     v_m2 = aux_haversine( trajectory[ targetPoint - 2, 2 ], trajectory[ targetPoint - 2, 1 ], trajectory[ targetPoint - 1, 2 ], trajectory[ targetPoint - 1, 1 ] )
#     s_m2 = v_m2 / ( trajectory[ targetPoint - 1, 0 ] - trajectory[ targetPoint - 2, 0 ] )
    
    
#     v_p1 = aux_haversine( trajectory[ targetPoint, 2 ], trajectory[ targetPoint, 1 ], trajectory[ targetPoint + 1, 2 ], trajectory[ targetPoint + 1, 1 ] )
#     s_p1 = v_p1 / ( trajectory[ targetPoint + 1, 0 ] - trajectory[ targetPoint, 0 ] )
    
    
#     v_p2 = aux_haversine( trajectory[ targetPoint + 1, 2 ], trajectory[ targetPoint + 1, 1 ], trajectory[ targetPoint + 2, 2 ], trajectory[ targetPoint + 2, 1 ] )
#     s_p2 = v_p2 / ( trajectory[ targetPoint + 2, 0 ] - trajectory[ targetPoint + 1, 0 ] )
    

#     return np.std( [ v_m2, v_m1, v_p1, v_p2 ] ) / np.mean( [ v_m2, v_m1, v_p1, v_p2 ] )



# def aux_haversine(lon1, lat1, lon2, lat2): # haversine is the angle distance between two points on a sphere 
    
#     from math import radians, cos, sin, asin, sqrt

#     R = 6372.8

#     dLat = radians(lat2 - lat1)
#     dLon = radians(lon2 - lon1)
#     lat1 = radians(lat1)
#     lat2 = radians(lat2)

#     a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
#     c = 2*asin(sqrt(a))

#     return R * c

In [5]:
globalSpeedWeight = 0.1
globalPower = 1.0

def aux_haversine(lon1, lat1, lon2, lat2): # haversine is the angle distance between two points on a sphere 
    
    from math import radians, cos, sin, asin, sqrt

    R = 6372.8

    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return R * c





def aux_CalculateErrorSinglePoint( trajectory, targetPoint ):
    
    segmDist = []
    segmSpeed = []
    for k in range( -2, 2 ):
        d = aux_haversine( trajectory[ targetPoint + k, 2 ], trajectory[ targetPoint + k, 1 ], \
                           trajectory[ targetPoint + k + 1, 2 ], trajectory[ targetPoint + k + 1, 1 ] )
        s = d / ( trajectory[ targetPoint + k + 1, 0 ] - trajectory[ targetPoint + k, 0 ] )
        segmDist.append( d )
        segmSpeed.append( s )
            
    segmSpeedD = np.array( segmSpeed )
    segmSpeedD = segmSpeedD[ 1: ] - segmSpeedD[ :-1 ]
    speedError = np.std( segmSpeedD ) / np.mean( segmSpeed )
    
    segmAngles = []
    for k in range( -1, 2 ):
        v1 = [ trajectory[ targetPoint + k + 1, 2 ] - trajectory[ targetPoint + k, 2 ], \
               trajectory[ targetPoint + k + 1, 1 ] - trajectory[ targetPoint + k, 1 ] ]
        v1 = v1 / np.linalg.norm( v1 )
        v2 = [ trajectory[ targetPoint + k, 2 ] - trajectory[ targetPoint + k - 1, 2 ], \
               trajectory[ targetPoint + k, 1 ] - trajectory[ targetPoint + k - 1, 1 ] ]
        v2 = v2 / np.linalg.norm( v2 )

        dot_product = np.dot( v1, v2 )
        angle = np.arccos( dot_product )
        segmAngles.append( angle / ( trajectory[ targetPoint + k + 1, 0 ] - trajectory[ targetPoint + k - 1, 0 ] ) )
        
    segmAnglesD = np.array( segmAngles )
    segmAnglesD = segmAnglesD[ 1: ] - segmAnglesD[ :-1 ]
    angleError = np.std( segmAnglesD ) / np.mean( segmAngles )
    
    speedError = np.power( speedError, globalPower )
    angleError = np.power( angleError, globalPower )
    
    totalError = globalSpeedWeight * speedError + ( 1.0 - globalSpeedWeight ) * angleError
#    if np.isnan( totalError ): totalError = 999.9
    
    return totalError

    




def aux_CalculateTrajectoryError( trajectory, variablePoints ):
    
    totalError = 0.0
    
    for k in range( 2, np.size( variablePoints ) - 2 ):
        
        if variablePoints[ k ]:
            single_point_error = aux_CalculateErrorSinglePoint( trajectory, k )
            
            if ~np.isnan(single_point_error):

                totalError += single_point_error
    
    return totalError

In [ ]:
def Algorithm_Hill_Climb( trajectory, variablePoints, auxParameters ):

    bestTraj = np.copy( trajectory )
    STD_DEV = .005


    for _ in range( auxParameters[ 'numIters' ] ):

        t1Traj = np.copy( bestTraj )

        for k in np.where(variablePoints)[0]:

            t2Traj = np.copy(t1Traj)

            neighbours = getNeighbours( t1Traj[k], n=2000, STD_DEV=STD_DEV )

            lowest_error = aux_CalculateTrajectoryError( t1Traj, variablePoints )

            for neighbour in neighbours:

                t2Traj[k + 1] = np.copy( neighbour )

                error = aux_CalculateTrajectoryError( t2Traj, variablePoints )

                if error < lowest_error:

                    t1Traj[k + 1] = np.copy( neighbour )

                    lowest_error = error

        bestTraj = np.copy(t1Traj)

    return bestTraj

In [11]:
def Algorithm_Local_Beam( originalTraj, variablePoints, auxParameters, beta=3 ):
    
    bestTraj = originalTraj
    Error = aux_CalculateTrajectoryError(originalTraj, variablePoints)

    # print(Error)

    for _ in range( auxParameters['numIters'] ):

        tier_to_paths = dict()

        for k in range(len(originalTraj)):

            tmp_path = np.copy(bestTraj)

            if variablePoints[k]:

                # vo tier_to_paths dictionary da inicijaliziram na tocka k u traektorijata prazna numpy.array

                tier_to_paths[k] = []

                # za sekoj variable point da izgeneriram novi points

                neighbours = getNeighbours(originalTraj[k])

                # za sekoj izgeneriran nov point da presmetam haversine distance

                for neighbour in neighbours:

                    tmp_path[k] = neighbour

                    error = aux_CalculateTrajectoryError(tmp_path, variablePoints)

                    if error < Error:

                        tier_to_paths[k].append(np.array([neighbour, error]))

                # najdobrite beta za k variable point da gi zacuvam 
                tier_to_paths[k] = sorted(tier_to_paths[k], key=lambda x: x[1], reverse=False)[:beta]
                
                print(tier_to_paths[k])
                print()

                if len(tier_to_paths[k]) > 0:
                    if tier_to_paths[k][0][1] < Error:
                        
                        print(tier_to_paths[k][0])

                        tmp_path[k] = tier_to_paths[k][0][0]

                        bestTraj = np.copy(tmp_path)
                        Error = tier_to_paths[k][0][1]

    return bestTraj

# Trajectory 1

In [7]:
targetPoints = []

targetPoints.append(np.array(range(260, 305)))
targetPoints.append(np.array(range(420, 470)))

targetPoints = np.hstack(targetPoints)

In [8]:
variablePoints = np.zeros( ( np.size( trajectory1, 0 ) ), dtype = bool )

In [9]:
for i in range(len(variablePoints)):
    if i in targetPoints:
        variablePoints[i] = True

In [12]:
Algorithm_Local_Beam(trajectory1, variablePoints, auxParameters)

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]



/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


[array([array([1.64955069e+09, 3.94399588e+01, 1.39530875e+02, 1.21920000e+04]),
       34.667295534763646], dtype=object), array([array([1.64955069e+09, 3.94048972e+01, 1.39510899e+02, 1.21920000e+04]),
       34.67519209152649], dtype=object), array([array([1.64955069e+09, 3.94623379e+01, 1.39474457e+02, 1.21920000e+04]),
       34.67780052569463], dtype=object)]

[array([1.64955069e+09, 3.94399588e+01, 1.39530875e+02, 1.21920000e+04])
 34.667295534763646]
[array([array([1.64955070e+09, 3.92266221e+01, 1.39645206e+02, 1.21843800e+04]),
       32.95171314897527], dtype=object), array([array([1.64955070e+09, 3.94105523e+01, 1.39264704e+02, 1.21843800e+04]),
       32.96211059310987], dtype=object), array([array([1.64955070e+09, 3.92560816e+01, 1.39617021e+02, 1.21843800e+04]),
       33.04343606874975], dtype=object)]

[array([1.64955070e+09, 3.92266221e+01, 1.39645206e+02, 1.21843800e+04])
 32.95171314897527]
[array([array([1.64955070e+09, 3.91002966e+01, 1.39396614e+02, 1.21920000e+0

[array([array([1.64955178e+09, 3.72698905e+01, 1.38722301e+02, 1.21843800e+04]),
       15.345092833685385], dtype=object), array([array([1.64955178e+09, 3.71620324e+01, 1.38745217e+02, 1.21843800e+04]),
       15.363523084986161], dtype=object), array([array([1.64955178e+09, 3.72692677e+01, 1.38685137e+02, 1.21843800e+04]),
       15.376550657721292], dtype=object)]

[array([1.64955178e+09, 3.72698905e+01, 1.38722301e+02, 1.21843800e+04])
 15.345092833685385]


array([[1.64954880e+09, 4.24550000e+01, 1.41428000e+02, 6.10362000e+03],
       [1.64954881e+09, 4.24516000e+01, 1.41419600e+02, 6.18744000e+03],
       [1.64954881e+09, 4.24478000e+01, 1.41410200e+02, 6.27126000e+03],
       ...,
       [1.64955400e+09, 3.47910000e+01, 1.36821600e+02, 1.60020000e+02],
       [1.64955401e+09, 3.47910000e+01, 1.36821600e+02, 1.60020000e+02],
       [1.64955402e+09, 3.47910000e+01, 1.36821600e+02, 1.60020000e+02]])

In [13]:
tmp = trajectory1[:, 1:3]

In [14]:
cTraj1 = correctedTraj1[:, 1:3]

NameError: name 'correctedTraj1' is not defined

In [ ]:
avg_lat = np.mean([row[0] for row in tmp])
avg_lon = np.mean([row[1] for row in tmp])

In [ ]:
m = folium.Map(
    location=[avg_lat, avg_lon],
    tiles="cartodbpositron",
    zoom_start=5
)

aline1 = folium.PolyLine(tmp, color='blue')
m.add_child(aline1)

aline2 = folium.PolyLine(cTraj1, color='red')
m.add_child(aline2)

m

In [ ]:
TIL.aux_CalculateTrajectoryError(trajectory1, variablePoints)

In [ ]:
TIL.aux_CalculateTrajectoryError(cTraj1, variablePoints)